In [8]:
# Adjusting current working directory to parent directory
from pathlib import Path
from os import chdir
from platform import system

try:
    current_directory
except: # First  run - initialize current_directory
    current_directory = Path.cwd()
    if system() == "Linux": # Colab
        from google.colab import drive
        drive.mount('/content/drive')
        current_directory = f"{current_directory}/drive/MyDrive/Colab Notebooks/RecTour2024Challenge"
    else:
        current_directory = current_directory.parent
finally:
    chdir(current_directory)



# External imports
import pandas as pd
import numpy as np

from random import randint

import torch
import torch.nn as nn

from sentence_transformers import SentenceTransformer

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model



# Internal imports
from src.data.csv_tools import csv_to_dataframe, dataframe_to_csv, save_submission
from src.data.pickle_tools import save_to_pickle, load_pickle
from src.data.keras_tools import save_keras_model, load_keras_model
from src.utils.preprocessing_tools import *

STEP 1 - Load data

In [9]:
test_users_df = csv_to_dataframe("test", "users")
users_embeddings_dict = load_pickle("test_user_embeddings_dict")
title_vectors_dict = load_pickle("test_title_vectors_dict")
positive_vectors_dict = load_pickle("test_positive_vectors_dict")
negative_vectors_dict = load_pickle("test_negative_vectors_dict")
accomodation_reviews_dict = load_pickle("test_reviews_grouped_by_accommodation")

STEP 2 - Load models

In [10]:
def create_contrastive_model():
    user_input = Input(shape=(12,), name='user')
    user_dense = Dense(384, activation='relu', name='user_dense')(user_input)

    review_input = Input(shape=(384,), name='review')

    cosine_similarity = Lambda(lambda tensors: tf.reduce_sum(tensors[0] * tensors[1], axis=-1, keepdims=True)) \
                              ([user_dense, review_input])

    return Model([user_input, review_input], cosine_similarity, name='contrastive_model')

In [11]:
title_model = create_contrastive_model()
title_model.load_weights("title_model.h5")
positive_model = create_contrastive_model()
positive_model.load_weights("positive_model.h5")
negative_model = create_contrastive_model()
negative_model.load_weights("negative_model.h5")

STEP 3 - Prepare the input to the models

In [12]:
def get_model_input(user_id, accommodation_id):
    user_embedding = users_embeddings_dict[user_id]
    accomodation_reviews_ids = accomodation_reviews_dict[accommodation_id]
    titles_embeddings = [title_vectors_dict[review_id] for review_id in accomodation_reviews_ids]
    positive_embeddings = [positive_vectors_dict[review_id] for review_id in accomodation_reviews_ids]
    negative_embeddings = [negative_vectors_dict[review_id] for review_id in accomodation_reviews_ids]
    user_embedding = np.array([user_embedding] * len(titles_embeddings))
    titles_embeddings = np.array(titles_embeddings)
    positive_embeddings = np.array(positive_embeddings)
    negative_embeddings = np.array(negative_embeddings)
    return [user_embedding, titles_embeddings], [user_embedding, positive_embeddings], [user_embedding, negative_embeddings]

In [13]:
# Create df - Structure: accommodation_id,user_id,review_1,review_2,review_3,...,review_10
columns = ["accommodation_id", "user_id"] + [f"review_{i}" for i in range(1, 11)]
submission_df = pd.DataFrame(columns=columns)

In [14]:
split_dfs = np.array_split(test_users_df, 20)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [15]:
type(test_users_df)

pandas.core.frame.DataFrame

In [ ]:
for current_index, current_test_users_df in enumerate(split_dfs[ :2]):
    current_submission_df = pd.DataFrame(columns=columns)
    for current_user_index, row in current_test_users_df.iterrows():
        user_id = row["user_id"]
        accomodation_id = row["accommodation_id"]
        title_input, positive_input, negative_input = get_model_input(user_id, accomodation_id)
        title_prediction = title_model.predict(title_input)
        positive_prediction = positive_model.predict(positive_input)
        negative_prediction = negative_model.predict(negative_input)
        # Get the top 10 reviews where the average is the highest
        top_10_reviews = []
        for i in range(len(title_prediction)):
            top_10_reviews.append((title_prediction[i] + positive_prediction[i] + negative_prediction[i]).mean())
        top_10_reviews = np.argsort(top_10_reviews)[-10:]
        reviews = [accomodation_reviews_dict[accomodation_id][i] for i in top_10_reviews]
        submission_df = submission_df._append(pd.Series([accomodation_id, user_id] + reviews, index=columns), ignore_index=True)

    dataframe_to_csv(submission_df, f"submission_part_{current_index}.csv")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
8/8 ━━━━━